# Journalist Classification with TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\gwehrm\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\gwehrm\AppData\Roaming\Python\Python36\s

import the dataset

In [2]:
articles = pd.read_csv("articles.csv")

Split into X and y

In [19]:
y = articles["author"]
X = articles["article"]

Define function next_batch (obtained stack_overflow)

In [4]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


Now, we've had all ingredients to build a logistic or softmax classification. Let's build a simple softmax classification.

In [20]:
# Importing necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# 80-20 splitting the dataset (80%->Training and 20%->Validation)

X_train, X_test, y_train, y_test = train_test_split(X, y
                                   ,test_size=0.2, random_state=1234)
y_train = np.array(y_train)
y_test = np.array(y_test)

# defining the bag-of-words transformer on the text-processed corpus # i.e., text_process() declared in II is executed...
bow_transformer=CountVectorizer(max_features = 5000).fit(X_train)
# transforming into Bag-of-Words and hence textual data to numeric..
text_bow_train=bow_transformer.transform(X_train).toarray()#ONLY TRAINING DATA

# transforming into Bag-of-Words and hence textual data to numeric..
text_bow_test=bow_transformer.transform(X_test).toarray()#TEST DATA

In [ ]:
tf.reset_default_graph()

n_inputs = 1 * 2500
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 58
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n_inputs], name='X')
y = tf.placeholder(tf.int64, shape=[None], name='y')

W = tf.Variable(tf.truncated_normal([n_inputs, n_outputs], stddev=0.02), name='weights')
b = tf.Variable(tf.zeros([n_outputs]), name='biases')

logits = tf.add(tf.matmul(X, W), b, name='logits')

with tf.name_scope('evaluation'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='xentropy')
    loss = tf.reduce_mean(xentropy, name='loss')
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

with tf.name_scope("train"):
    grad_W, grad_b = tf.gradients(loss, [W, b])
    update_W = tf.assign(W, W - learning_rate * grad_W)
    update_b = tf.assign(b, b - learning_rate * grad_b)

init = tf.global_variables_initializer()

In [ ]:
n_epochs = 200
batch_size = 100

with tf.Session() as sess:
    init.run()
    print("Epoch\tTrain accuracy\tTest accuracy")
    for epoch in range(n_epochs):
        for iteration in range(len(text_bow_train) // batch_size):
            X_batch, y_batch = next_batch(batch_size,text_bow_train,y_train)
            sess.run([update_W, update_b], feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: text_bow_test, y: y_test})
        print('%d\t%f\t%f' % (epoch, acc_train, acc_test))
    classification = tf.run(y, feed_dict)
    print(classification)
        

The accuracy was not bad, roughly 64%. Compared to the 49% from the Naive Bayes, this is a tremendeous increase. However, Now let's try it with tensorflow

In [21]:
import tensorflow as tf
tf.reset_default_graph()

n_inputs = 1 * 5000
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 58
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n_inputs], name='X')
y = tf.placeholder(tf.int64, shape=[None], name='y')

W = tf.Variable(tf.truncated_normal([n_inputs, n_outputs], stddev=0.02), name='weights')
b = tf.Variable(tf.zeros([n_outputs]), name='biases')

logits = tf.add(tf.matmul(X, W), b, name='logits')

with tf.name_scope('evaluation'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='xentropy')
    loss = tf.reduce_mean(xentropy, name='loss')
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [22]:
n_epochs = 200
batch_size = 50

with tf.Session() as sess:
    init.run()
    print("Epoch\tTrain accuracy\tTest accuracy")
    for epoch in range(n_epochs):
        for iteration in range(len(text_bow_train) // batch_size):
            X_batch, y_batch = next_batch(batch_size,text_bow_train,y_train)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: text_bow_test, y: y_test})
        print('%d\t%f\t%f' % (epoch, acc_train, acc_test))

Epoch	Train accuracy	Test accuracy
0	0.420000	0.273148
1	0.480000	0.310185
2	0.560000	0.351852
3	0.620000	0.416667
4	0.720000	0.444444
5	0.560000	0.458333
6	0.720000	0.472222
7	0.840000	0.472222
8	0.900000	0.490741
9	0.780000	0.504630
10	0.820000	0.523148
11	0.900000	0.523148
12	0.940000	0.541667
13	0.900000	0.541667
14	0.920000	0.537037
15	0.960000	0.564815
16	0.940000	0.569444
17	0.920000	0.555556
18	0.960000	0.560185
19	0.820000	0.583333
20	0.900000	0.583333
21	0.940000	0.587963
22	0.880000	0.574074
23	0.940000	0.597222
24	0.860000	0.601852
25	0.940000	0.587963
26	0.940000	0.597222
27	0.980000	0.601852
28	0.960000	0.601852
29	0.940000	0.620370
30	0.940000	0.620370
31	0.960000	0.620370
32	0.980000	0.620370
33	0.960000	0.620370
34	0.900000	0.625000
35	0.900000	0.615741
36	0.940000	0.629630
37	0.960000	0.634259
38	0.980000	0.629630
39	0.940000	0.634259
40	0.920000	0.629630
41	0.940000	0.643519
42	0.980000	0.652778
43	0.940000	0.634259
44	0.960000	0.648148
45	1.000000	0.643519
46	1.0000

Another Optimizer:

In [23]:
import tensorflow as tf
tf.reset_default_graph()

n_inputs = 1 * 5000
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 58
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=[None, n_inputs], name='X')
y = tf.placeholder(tf.int64, shape=[None], name='y')

W = tf.Variable(tf.truncated_normal([n_inputs, n_outputs], stddev=0.02), name='weights')
b = tf.Variable(tf.zeros([n_outputs]), name='biases')

logits = tf.add(tf.matmul(X, W), b, name='logits')

with tf.name_scope('evaluation'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='xentropy')
    loss = tf.reduce_mean(xentropy, name='loss')
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

Train the model

In [ ]:
n_epochs = 200
batch_size = 50

with tf.Session() as sess:
    init.run()
    print("Epoch\tTrain accuracy\tTest accuracy")
    for epoch in range(n_epochs):
        for iteration in range(len(text_bow_train) // batch_size):
            X_batch, y_batch = next_batch(batch_size,text_bow_train,y_train)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: text_bow_test, y: y_test})
        print('%d\t%f\t%f' % (epoch, acc_train, acc_test))
    confusion = tf.confusion_matrix(labels=y_, predictions=y, num_classes=num_classes)
    print(confusion)
    

Epoch	Train accuracy	Test accuracy
0	0.920000	0.504630
1	0.900000	0.583333
2	0.980000	0.620370
3	0.960000	0.569444
4	1.000000	0.583333
5	1.000000	0.643519
6	1.000000	0.638889
7	1.000000	0.662037
8	1.000000	0.662037
9	1.000000	0.652778
10	1.000000	0.643519
11	1.000000	0.652778
12	1.000000	0.666667
13	1.000000	0.662037
14	1.000000	0.652778
15	1.000000	0.652778
16	1.000000	0.666667
17	1.000000	0.666667
18	1.000000	0.657407
19	1.000000	0.638889
20	1.000000	0.648148
21	1.000000	0.648148
22	1.000000	0.643519
23	1.000000	0.634259
24	1.000000	0.634259
25	1.000000	0.657407
26	1.000000	0.657407
27	1.000000	0.643519
28	1.000000	0.657407
29	1.000000	0.648148
30	1.000000	0.648148
31	1.000000	0.657407
32	1.000000	0.652778
33	1.000000	0.652778
34	1.000000	0.648148
35	1.000000	0.652778
36	1.000000	0.662037
37	1.000000	0.652778
38	1.000000	0.662037
39	1.000000	0.657407
40	1.000000	0.638889
41	1.000000	0.643519
42	1.000000	0.643519
43	1.000000	0.652778
44	1.000000	0.643519
45	1.000000	0.648148
46	1.0000